In [1]:
from openai import OpenAI
import json, os, sys
import os
import re
import random

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
def generate_difference(chunk_a, chunk_b, client, model="gpt-4o"):
    prompt = f"""
You are an expert in news analysis and skilled at creating multiple-choice question answers that help to differentiate topics between articles. 
I have some summaries of news articles and I need to generate unique, comprehensive multiple-choice questions based on these summaries.

Your task is to generate questions that:
- Help to differentiate topics between the articles.
- Are common to all articles, not specific to any one article.
- Are unique and not repeated in any other batch.
- Include "None of the above" as one of the options for each question.

Below is the format and example I need:

1. What was done?
   - A. The Chinese coast guard seized one of four food packs dropped by a Philippine military plane for Filipino navy personnel at a territorial outpost. After discovering the package contained food, they dumped it into the sea.
   - B. Philippine soldiers were reported to have pointed guns at Chinese coast guard personnel during a resupply mission to the grounded Sierra Madre ship. The Chinese coast guard responded to the resupply operation, which included food drops, by observing armed Philippine soldiers on the ship’s deck.
   - C. Turkish Foreign Minister Hakan Fidan began a trip to China and expressed priorities to support Hamas against Israel and increase trade with China, without condemning the Uyghur genocide.
   - D. The Philippines is collaborating with the United States and Japan to ensure the West Philippine Sea (WPS) remains free and safe amid tensions with Chinese maritime forces.
   - E. None of the above

Below is the questions that need multiple choice answers to be generated:

1. What was done?
2. Who was involved?
3. Where did the event take place?
4. When did the event occur?
5. Why did the event happen?
6. How was the event carried out?
7. What was the outcome?
8. What was the primary motive behind the actions?
9. What was the reaction of the involved parties?
10. What was the broader context of the event?

Please generate as many unique and comprehensive answers as possible based on the given article summaries below. 
Make sure each answer is designed to highlight differences between the articles.

Summaries: 
{chunk_b}
{chunk_a}

Generate as many as you can unique questions based on the above summaries.



"""
    messages = [{"role": "user", "content": prompt}]
    try:
        response = client.chat.completions.create(
            model=model, messages=messages, temperature=0
        )

        content = response.choices[0].message.content
        return content
    except Exception as e:  # if the model fails to return a response
        print(f"Error: {e}")
        return "Sorry, error from GPT."

In [4]:
def process_batches(
    input_file_a, input_file_b, output_file, client, model, label_a_index, label_b_index
):
    # Read the article from the input JSON file
    with open(input_file_a, "r") as f:
        articles = json.load(f)

    # Extract summaries from the articles
    article_body = [article["summary"] for article in articles]

    # get the article body with the label_a_index list
    article_a = [article_body[i] for i in label_a_index]

    # convert the article to JSON string
    chunk_a = json.dumps(article_a)

    with open(input_file_b, "r") as f:
        articles = json.load(f)

    # Extract summaries from the articles
    article_body = [article["summary"] for article in articles]

    # get the article body with the label_a_index list
    article_b = [article_body[i] for i in label_b_index]

    # convert the article to JSON string
    chunk_b = json.dumps(article_b)

    responses = []

    difference = generate_difference(chunk_a, chunk_b, client, model=model)
    responses.append(
        {
            "Difference": difference,
        }
    )

    # Write the generated questions to the output JSON file
    with open(output_file, "w") as f:
        json.dump(responses, f, indent=4)

In [5]:
import pandas as pd

# read the label file
# labels = pd.read_csv("labels_4o.csv", header=None, names=["label"])
label = list(range(200))
# get the label index if labe is 1
# label = list(labels[labels["label"] != 1].index)
# only get the number smaller than 100
label_a_index = [i for i in label if i < 100]
label_b_index = [i for i in label if i >= 100]
# add 100 to each index
label_b_index = [i - 100 for i in label_a_index]

In [6]:
input_file_a = "summary_Russo_Ukrainian_War_eng.json"
input_file_b = "summary_Russo_Ukrainian_War_zho.json"
output_file = "EDA.json"

# print(label_a_index)
# Assuming you have your OpenAI API client initialized as `client`
process_batches(
    input_file_a,
    input_file_b,
    output_file,
    client,
    model="gpt-4o",
    label_a_index=label_a_index,
    label_b_index=label_b_index,
)

In [7]:
# print the value from JSON file

with open(output_file, "r") as f:
    output = json.load(f)[0]
    print(output["Difference"])

1. What was done?
   - A. The U.S. announced a $150 million military aid package for Ukraine, including HAWK missiles and ammunition for HIMARS systems.
   - B. Ukrainian President Volodymyr Zelenskyy urged former U.S. President Donald Trump to reveal his plan for ending the Russia-Ukraine war.
   - C. The European Commission's Directorate-General for Competition published a report on protecting competition in the EU.
   - D. The U.S. Supreme Court issued a 6-3 opinion granting former President Donald Trump broad immunity for official acts.
   - E. None of the above

2. Who was involved?
   - A. Ukrainian President Volodymyr Zelenskyy and former U.S. President Donald Trump.
   - B. Turkish President Recep Tayyip Erdogan and Russian President Vladimir Putin.
   - C. Chinese President Xi Jinping and Russian President Vladimir Putin.
   - D. U.S. Secretary of Defense Lloyd J. Austin III and Ukrainian Defense Minister Rustem Umerov.
   - E. None of the above

3. Where did the event take pl

1. What was done?
   - A. The U.S. announced a $150 million military aid package for Ukraine, including HAWK missiles and ammunition for HIMARS systems.
   - B. Ukrainian President Volodymyr Zelenskyy urged former U.S. President Donald Trump to reveal his plan for ending the Russia-Ukraine war.
   - C. The European Commission's Directorate-General for Competition published a report on protecting competition in the EU.
   - D. The U.S. Supreme Court issued a 6-3 opinion granting former President Donald Trump broad immunity for official acts.
   - E. None of the above

2. Who was involved?
   - A. Ukrainian President Volodymyr Zelenskyy and former U.S. President Donald Trump.
   - B. Turkish President Recep Tayyip Erdogan and Russian President Vladimir Putin.
   - C. Chinese President Xi Jinping and Russian President Vladimir Putin.
   - D. U.S. Secretary of Defense Lloyd J. Austin III and Ukrainian Defense Minister Rustem Umerov.
   - E. None of the above

3. Where did the event take place?
   - A. Astana, Kazakhstan
   - B. Kyiv, Ukraine
   - C. Washington, DC, USA
   - D. Brussels, Belgium
   - E. None of the above

4. When did the event occur?
   - A. July 3, 2023
   - B. June 27, 2023
   - C. May 2024
   - D. July 4, 2024
   - E. None of the above

5. Why did the event happen?
   - A. To enhance Ukraine's air defenses and overall military capabilities.
   - B. To discuss bilateral ties and regional cooperation.
   - C. To address the severe impact of Russian attacks on Ukrainian civilians.
   - D. To provide insights into the development and future direction of generative AI technology.
   - E. None of the above

6. How was the event carried out?
   - A. By announcing a new security assistance package under the Presidential Drawdown Authority.
   - B. By holding a meeting at the Pentagon to announce a military assistance package.
   - C. By publishing a report on competition in the EU.
   - D. By issuing a Supreme Court opinion.
   - E. None of the above

7. What was the outcome?
   - A. The U.S. provided Ukraine with air defense interceptors, artillery, and anti-tank weapons.
   - B. Ukrainian President Volodymyr Zelenskyy expressed willingness to meet with Donald Trump.
   - C. The European Commission identified key areas for future competition policy.
   - D. The U.S. Supreme Court ruling potentially empowered Trump's political comeback.
   - E. None of the above

8. What was the primary motive behind the actions?
   - A. To support Ukraine against Russian aggression.
   - B. To facilitate peace negotiations between Russia and Ukraine.
   - C. To enhance competition and innovation in the EU.
   - D. To grant immunity for official acts to a former president.
   - E. None of the above

9. What was the reaction of the involved parties?
   - A. Ukrainian President Volodymyr Zelenskyy urged Trump to reveal his plan for ending the war.
   - B. Kremlin spokesman Dmitry Peskov dismissed the possibility of Erdogan serving as an intermediary.
   - C. Chinese President Xi Jinping emphasized the importance of China-Russia relations.
   - D. Justice Sonia Sotomayor dissented, warning of extreme scenarios.
   - E. None of the above

10. What was the broader context of the event?
    - A. The ongoing Russia-Ukraine conflict and international support for Ukraine.
    - B. The geopolitical tensions involving China, Russia, and the West.
    - C. The EU's efforts to protect competition in a changing world.
    - D. The U.S. political landscape and the implications of presidential immunity.
    - E. None of the above

### Summary of Common Answers:

#### 1. What was done?
- **Military Aid and Support**: The U.S. announced multiple military aid packages for Ukraine, including air defense interceptors, artillery, and anti-tank weapons. NATO allies also committed significant financial and military support.
- **Diplomatic Efforts and Summits**: Various diplomatic meetings and summits took place, including the NATO Summit in Washington, the Shanghai Cooperation Organization (SCO) summit in Kazakhstan, and bilateral meetings between leaders like Xi Jinping and Vladimir Putin.
- **Legislative and Policy Changes**: Russia tightened its draft laws to close loopholes, and the U.S. Supreme Court issued a ruling granting former President Trump broad immunity for official acts.
- **Economic and Trade Agreements**: Turkmenistan and Iran signed a gas delivery contract, and CoinDCX acquired BitOasis to expand its presence in the MENA region.

#### 2. Who was involved?
- **Political Leaders**: Key figures included U.S. President Joe Biden, Ukrainian President Volodymyr Zelensky, Russian President Vladimir Putin, Chinese President Xi Jinping, and Turkish President Recep Tayyip Erdogan.
- **International Organizations**: NATO, the European Union, the United Nations, and the Shanghai Cooperation Organization (SCO) were prominently involved.
- **Military and Defense Officials**: U.S. Defense Secretary Lloyd Austin, Ukrainian Defense Minister Rustem Umerov, and various military personnel from NATO and allied countries.
- **Business and Economic Entities**: Companies like CoinDCX, BitOasis, and Dark Star Minerals Inc. were involved in significant economic and trade activities.

#### 3. Where did the event take place?
- **Ukraine and Russia**: The ongoing conflict and related military activities were centered in Ukraine and Russia, with significant events in regions like Kharkiv, Crimea, and the Black Sea.
- **Washington, DC**: The NATO Summit and various diplomatic meetings took place in Washington, DC.
- **Kazakhstan**: The SCO summit was held in Astana, Kazakhstan, with discussions on regional cooperation and geopolitical issues.
- **Middle East and North Africa (MENA)**: Economic activities and agreements, such as the acquisition of BitOasis by CoinDCX, were focused in the MENA region.

#### 4. When did the event occur?
- **July 2023**: Many events, including the NATO Summit, the SCO summit, and various military aid announcements, occurred in early July 2023.
- **Ongoing Conflict**: The Russia-Ukraine war, which has been ongoing since February 2022, continued to be a central focus.
- **Recent Legislative Changes**: Russia's new draft laws were set to take effect on November 1, 2024.
- **Economic Agreements**: The Turkmenistan-Iran gas contract and CoinDCX's acquisition of BitOasis were finalized in early July 2023.
...
- **Legislative and Policy Changes**: New laws and policies, particularly in Russia, indicate preparations for prolonged conflict and efforts to tighten control over conscription and military resources.

### Conclusion:
The provided articles collectively illustrate a world grappling with ongoing military conflicts, significant diplomatic efforts, and strategic economic agreements. The involvement of key political leaders and international organizations underscores the global nature of these issues, with far-reaching implications for international stability and security.
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...

### Summary of Common Answers:

#### 1. What is the main theme of the article?
1. The ongoing Russia-Ukraine conflict and its geopolitical implications.
2. International military aid and support for Ukraine.
3. Political dynamics and leadership changes in various countries.
4. Economic and technological developments influenced by geopolitical tensions.

#### 2. What are the primary keywords or phrases in this article?
1. "Russia-Ukraine conflict," "military aid," "NATO."
2. "Geopolitical tensions," "sanctions," "economic impact."
3. "Ceasefire," "sovereignty," "territorial integrity."
4. "Defense systems," "air defense," "Patriot missiles."

#### 3. What event or issue is the article centered around?
1. The Russia-Ukraine war and its impact on global politics.
2. The provision of military aid and defense systems to Ukraine.
3. Political maneuvers and leadership changes in response to the conflict.
4. Economic and technological shifts due to geopolitical tensions.

#### 4. Which region or country is primarily discussed in the article?
1. Ukraine.
2. Russia.
3. United States.
4. European Union.

#### 5. Who is most affected by the issues discussed in the article?
1. Ukrainian civilians and military.
2. Russian military and political leadership.
3. International allies and partners of Ukraine.
4. Global economic and political stakeholders.

#### 6. What type of sources does the article cite?
1. Government reports and official statements.
2. Expert opinions and analysis.
3. International organizations and think tanks.
4. Media reports and news agencies.

#### 7. What is the geopolitical significance of the article’s content?
1. The ongoing conflict in Ukraine as a major global security issue.
2. The role of international alliances like NATO in supporting Ukraine.
3. The impact of the conflict on global economic stability and security.
4. The strategic maneuvers of global powers in response to the conflict.

#### 8. What are the potential future implications mentioned in the article?
1. Continued military and economic support for Ukraine.
2. Potential shifts in global alliances and power dynamics.
3. Long-term geopolitical instability and economic repercussions.
4. Possible changes in international policies and defense strategies.

### Detailed Analysis:

#### 1. What is the main theme of the article?
- **Russia-Ukraine Conflict**: Many articles focus on the ongoing war, its developments, and its broader geopolitical implications.
- **Military Aid**: Several articles discuss the provision of military aid to Ukraine, including specific defense systems and financial support.
- **Political Dynamics**: Articles highlight political changes and leadership decisions in various countries in response to the conflict.
- **Economic and Technological Developments**: Some articles address how geopolitical tensions are influencing economic policies and technological advancements.

#### 2. What are the primary keywords or phrases in this article?
- **Conflict and Aid**: Keywords like "Russia-Ukraine conflict," "military aid," and "NATO" are prevalent.
- **Geopolitical Tensions**: Terms such as "sanctions," "economic impact," and "geopolitical tensions" are frequently mentioned.
- **Ceasefire and Sovereignty**: Phrases like "ceasefire," "sovereignty," and "territorial integrity" are common in discussions about potential resolutions.
- **Defense Systems**: Keywords related to military technology, such as "defense systems," "air defense," and "Patriot missiles," are often highlighted.

#### 3. What event or issue is the article centered around?
- **Russia-Ukraine War**: The central focus is the ongoing conflict and its various dimensions.
- **Military Aid**: Specific articles center around the provision and impact of military aid to Ukraine.
- **Political Maneuvers**: Leadership changes and political decisions in response to the conflict are key issues.
- **Economic Shifts**: The impact of the conflict on global economic and technological landscapes is also a significant focus.

#### 4. Which region or country is primarily discussed in the article?
- **Ukraine**: The primary geographical focus is on Ukraine.
- **Russia**: Russia is also a central focus, particularly in terms of its actions and responses.
- **United States**: The role of the U.S. in providing aid and its political stance is frequently discussed.
- **European Union**: The EU's involvement and its internal political dynamics are also highlighted.

#### 5. Who is most affected by the issues discussed in the article?
- **Ukrainian Civilians and Military**: The direct impact on Ukraine's population and armed forces is a major concern.
- **Russian Leadership**: The implications for Russia's military and political leadership are significant.
- **International Allies**: Countries supporting Ukraine, particularly NATO members, are affected.
- **Global Stakeholders**: The broader global community, including economic and political stakeholders, is impacted.

#### 6. What type of sources does the article cite?
- **Government Reports**: Official statements and reports from governments are commonly cited.
- **Expert Opinions**: Analysis and opinions from experts and analysts are frequently referenced.
- **International Organizations**: Reports and statements from international bodies and think tanks are used.
- **Media Reports**: News agencies and media outlets provide much of the information.

#### 7. What is the geopolitical significance of the article’s content?
- **Global Security Issue**: The conflict is a major global security concern.
- **Role of Alliances**: The involvement of NATO and other international alliances is crucial.
- **Economic Impact**: The conflict's impact on global economic stability is significant.
- **Strategic Maneuvers**: The strategic responses of global powers are important for understanding the broader implications.

#### 8. What are the potential future implications mentioned in the article?
- **Continued Support for Ukraine**: Ongoing military and economic aid is expected.
- **Shifts in Alliances**: Potential changes in global alliances and power dynamics are anticipated.
- **Geopolitical Instability**: Long-term instability and economic repercussions are likely.
- **Policy Changes**: Possible adjustments in international policies and defense strategies are discussed.

Based on the provided articles, here are the four most common answers to each of the questions:

1. **What is the main theme of the article?**
   - The ongoing Russia-Ukraine conflict and its geopolitical implications.
   - International diplomatic efforts and military aid related to the Russia-Ukraine war.
   - The role of major global powers (e.g., the U.S., China, NATO) in the conflict.
   - Political and economic impacts of the war on various countries and regions.

2. **What are the primary keywords or phrases in this article?**
   - Russia-Ukraine conflict
   - Military aid
   - Geopolitical tensions
   - Diplomatic efforts
   - NATO
   - Sanctions
   - Peace negotiations
   - Defense systems

3. **What event or issue is the article centered around?**
   - The Russia-Ukraine war and its various military, political, and economic dimensions.
   - Diplomatic visits and meetings related to the conflict.
   - Military aid packages and defense strategies.
   - International reactions and alliances forming in response to the war.

4. **Which region or country is primarily discussed in the article?**
   - Ukraine
   - Russia
   - United States
   - European Union (including specific countries like Hungary, Germany, and France)

5. **Who is most affected by the issues discussed in the article?**
   - Ukrainian civilians and military personnel.
   - Russian military and political leadership.
   - International allies and partners of Ukraine (e.g., NATO countries).
   - Global economic and political stakeholders.

6. **What type of sources does the article cite?**
   - Official government statements and press releases.
   - Reports from international organizations (e.g., NATO, UN).
   - Statements from political leaders and diplomats.
   - Military and defense analysis from experts and think tanks.

7. **What is the geopolitical significance of the article's content?**
   - The shifting balance of power in Eastern Europe and its global implications.
   - The role of international alliances and partnerships in the conflict.
   - The impact of military aid and sanctions on the war's progression.
   - The broader strategic interests of major powers like the U.S., China, and Russia.

8. **What are the potential future implications mentioned in the article?**
   - Prolonged conflict and continued military engagements.
   - Changes in international alliances and defense strategies.
   - Potential for escalated tensions or peace negotiations.
   - Economic and political shifts resulting from the war and international responses.